# Chapter 02: Object-Oriented Programming


## 2.5 Namespaces and Object-Orientation
A **namespace** is an abstraction that manages all of the identifiers that are defined in a particular scope, mapping each name to its associated value. In Python, functions, classes, and modules are all first-class objects, and so the "value" associated with an identifier in a namespace may in fact be a function, class, or module.


### Instance and Class namespaces
We begin by exploring what is known as the **instance namespace**, which manages attributes specific to an individual object. For example, each instance of our `CreditCard` class maintains a distinct balance, a distinct account number, a distinct credit limit, and so on. Each credit card will have a dedicated instance namespace to manage such values.

There is a separate **class namespace** for each class that has been defined. This namespace is used to manage members that are to be *shared* by all instances of a class, or used without reference to any particular instance. For example, `make_payment` method of the `CreditCard` class is not stored independently by each instance of that class. That member function is stored within the namespace of the `CreditCard` class. Our `Predatory CreditCard` class has its own namespace, containing the three methods we defined for that subclass: `__init__`, `charge` and `process_month`.

#### How Entries Are Established in a Namespace
When inheritance is used, there is still a single *instance namespace* per object.

A *class namespace* includes all declarations that are made directly within the body of the class definition.

#### Class Data Members
A class-level data member is often used when there is some value, such as a constant, that is to be shared by all instances of a class. In such a case, it would be unnecessarily wasteful to have each instance store that value in its instance namespace.

```python
class PredatoryCreditCard(CreditCard):
    OVERLIMIT_FEE = 5    # this is a class-level member
    
    def charge(self, price):
        success = super().charge(price)
        if not success:
            self._balance += PredatoryCreditCard.OVERLIMIT_FEE
        return success
```

The data member, `OVERLIMIT_FEE`, is entered into the `PredatoryCreditCard` class namespace because that assignment takes place within the immediate scope of the class definition, and without any qualifying identifier.

#### Nested Classes
It is also possble to nest one class definition within the scope of another class. This is a useful construct, which we wil exploit several times in this book in the implementation of data structures. This can be done by using a syntax such as

```python
class A:
    class B:
```

In this case, class `B` is the nested class. The identifier `B` is entered in to the namespace of class `A` associated with the newly defined class. We note that this technique is unrelated to the concept of inheritance, as class `B` does not inherit from class `A`.

Nesting one class in the scope of another makes clear that the nested class exists for support of the outer class. Furthermore, it can help reduce potential name conflicts, because it allows for a similarly named class to exist in another context. For example we will later introduce a data structure known as a **linked list** and will define a nested node class to store the individual components of the list. We will also introduce a data sturcture known as a **tree** that depends upon its own nested node class. Thes e two structures rely on different node definitions, and by nesting those within the respective container classes, we avoid ambiguity.

Another advantage of one class being nested as a member of another is taht it allows for a more advanced form of inheritance in which a subclass of the outer class overrides the definition of its nested class.

#### Dictionaries and the `__slots__` Declaration
By default, Python represents each namespace with an instance of the built-in `dict` class that maps identifying names in that scope to the associated objects. While a dictionary structure supports relatively efficient anme lookups, it requires additional memory usage beyond the raw data that it stores.

Python provides a more direct mechnism for representing instance namespaces that avoids the use of an auxiliary dictionary. To use the streamlined representation for all instances of a class, that class definition must provide a class-levle member named `__slots__` that is assigned to a fixed sequene of strings that serve as names fro instance variables. For example, with our `CreditCard` class, we would declare the following:

```python
class CreditCard:
    __slots__ = '_customer', '_bank', '_account', '_balance', '_limit'
```

In this example, the righ-hand side of the assignment is technically a tuple.

When inheritance is used, if the base class declares `__slots__`, a subclass must also declare `__slots__` to avoid creation of instance dictionaries. The declaration in the subclass should only include names of supplemental methods that are newly introduced. For example, our `PredatoryCreditCard` declaration would include the following declaration:

```python
class PredatoryCreditCard(CreditCard):
    __slots__ = '_apr'  # in addition to the inherited members
```

#### Name Resolution and Dynamic Dispatch
In this section, we examine the process that is used when *retrieving* a name in Python's object-oriented framework. When the dot operator syantax is used to access an existing number, such as `obj.foo`, the Python interpreter begins a name resolution process, described as follows:

1. The instance namespace is searched; if the desired name is found, its associated value is used.
2. Otherwise the class namespace, for the class to which the instance belongs, is searched; if the name is found, its associated value is used.
3. If the name was not found in the immediate class namespace, the search continues upward through the inheritance hierarchy, checking the class namespace for each ancestor (commonly checking the superclass, then its superclass and so on). The first time the name is found, its associate value is used.
4. If the name has still not found, an `AttributeError` is raised.

In traditional object-oriented terminology, Python uses what is known as **dynamic dispatch** (or **dynamic binding**) to determine, at run-time, which iplementation of a function to call based upon the type of the object upon which it is invoked. This is contrast to some languages that use **static dispatching**, making a compile-time decision as to which version of a function to call, based upon the declared type of a variable.